In [1]:
import sys
project_path = r"D:/DaiHoc/machinelearning/TLCN/DoAnTotNghiep_chat_bot/"
sys.path.append(project_path)
from source.function.utils_result import RAG
from source.search.utils_search import Qdrant_Utils
from source.rerank.utils_rerank import Rerank_Utils  
from source.model.embedding_model import Sentences_Transformer_Embedding
from source.model.extract_model import Bert_Extract
from source.model.generate_model import Gemini
from source.model.rerank_model import Cohere
from source.data.vectordb.qdrant import Qdrant_Vector
from source.core.config import Settings
from source.model.rerank_model_finetune import RerankModelFinetune
from source.generate.generate import Gemini_Generate
from source.extract.utils_extract import Extract_Information
from rouge_score import rouge_scorer
from sklearn.metrics import f1_score
import cohere
import numpy as np
import pandas as pd
from tqdm import tqdm
setting=Settings()
gemini=Gemini(setting)
print(setting.RERANK)
cohere_api=Cohere(setting)
bert=Bert_Extract(setting)
rerank_model_fintuned=RerankModelFinetune(setting)
sentences_transformer_embedding=Sentences_Transformer_Embedding(setting)
qdrant=Qdrant_Vector(setting,sentences_transformer_embedding)
rerank_Utils=Rerank_Utils(cohere_api,rerank_model_fintuned)
extract_Utils= Extract_Information(bert)
generate_Utils=Gemini_Generate(gemini,setting)
qdrant_Utils=Qdrant_Utils(qdrant, generate_Utils)
rag=RAG(generate_Utils,extract_Utils,qdrant_Utils,rerank_Utils,setting,sentences_transformer_embedding)
import pandas as pd
from tqdm import tqdm
import google.generativeai as genai 
from sklearn.metrics.pairwise import cosine_similarity
import os

hghaan/rerank_model


## 1.Eval Gemini model

In [2]:
# Đọc file CSV chứa câu hỏi và câu trả lời đã có (nếu có)
df = pd.read_csv('./data/data_processed/final_data_system_response.csv')

In [3]:

pattern = r'Xin\s+lỗi\s+bạn'  # regex cho “Xin lỗi bạn” (có thể có nhiều khoảng trắng)

# Chỉ giữ lại những dòng không khớp pattern
df = df[~df['answer_from_gemini_rag_basic'].str.contains(pattern, regex=True, na=False)]

In [4]:
df = df.reset_index(drop=True)


In [ ]:
def Generate_Response(df_batch):
    answers = []
    for idx, question in tqdm(list(enumerate(df_batch['question'])), desc="Sinh trả lời"):
        try:
            article_doc = rag.get_gemini_response_rag_final(question)
            answers.append(article_doc)
        except Exception as e:
            print(f"\n❌ Lỗi tại index {idx} (DataFrame index = {df_batch.index[idx]}):\n{question}")
            answers.append("")
    return answers


df['answer_from_gemini_rag_final'] = Generate_Response(df)


Sinh trả lời:  12%|█▏        | 65/564 [16:20<1:56:48, 14.05s/it]

DEBUG - raw_text:
 {
  \"answer\": \"Hành vi của anh M đối với chị D có thể bị xử lý theo pháp luật về phòng, chống bạo lực gia đình và các quy định khác liên quan.\n\n*   Hành vi mắng chửi, đánh đập chị D của anh M là hành vi bạo lực gia đình và có thể bị xử phạt hành chính. Cụ thể, việc sử dụng lời lẽ lăng mạ, xúc phạm danh dự, nhân phẩm của vợ có thể bị phạt cảnh cáo hoặc phạt tiền.\n*   Nếu hành vi của anh M gây ra thương tích hoặc tổn hại đến sức khỏe của chị D, anh M có thể bị truy cứu trách nhiệm hình sự.\n*   Ngoài ra, chị D có quyền yêu cầu Tòa án hoặc cơ quan có thẩm quyền áp dụng các biện pháp ngăn chặn và xử lý hành vi bạo lực của anh M.\n\nNhư vậy, hành vi của anh M có thể bị xử phạt hành chính hoặc truy cứu trách nhiệm hình sự tùy thuộc vào mức độ nghiêm trọng của hành vi và hậu quả gây ra.\",
  \"key\": [
    \"0\",
    \"4\",
    \"3\",
    \"1\",
    \"2\"
  ]
}

❌ Lỗi tại index 64 (DataFrame index = 64):
Câu 2. Anh M và chị D lấy nhau đã được gần 10 năm. Thời gian gần

Sinh trả lời:  14%|█▍        | 81/564 [20:48<2:09:40, 16.11s/it]

DEBUG - raw_text:
 {
  "answer": "Chào bạn, dựa trên thông tin bạn cung cấp, đây là các bước bạn nên thực hiện khi phát hiện trẻ bị bỏ rơi:\n\n*   **Thông báo và lập biên bản:** Gia đình bạn cần thông báo cho Ủy ban nhân dân cấp xã nơi chị bạn phát hiện ra cháu bé. Ủy ban nhân dân cấp xã sẽ có trách nhiệm lập biên bản về việc trẻ bị bỏ rơi. Biên bản này cần có chữ ký của người lập (đại diện Ủy ban nhân dân cấp xã), người phát hiện trẻ (chị gái bạn), và người làm chứng (nếu có). Biên bản này sẽ được lưu giữ tại Ủy ban nhân dân cấp xã và một bản giao cho gia đình bạn để tạm thời nuôi dưỡng cháu bé.\n*   **Niêm yết thông tin:** Sau khi lập biên bản, Ủy ban nhân dân cấp xã sẽ niêm yết thông tin về việc trẻ bị bỏ rơi tại trụ sở trong vòng 7 ngày liên tục.\n*   **Đăng ký khai sinh:** Nếu hết thời hạn niêm yết mà không có thông tin về cha mẹ đẻ của trẻ, Ủy ban nhân dân cấp xã sẽ thông báo cho gia đình bạn để tiến hành đăng ký khai sinh cho trẻ.\n\n**Về việc đăng ký khai sinh:**\n\n*   Gia đìn

Sinh trả lời:  40%|███▉      | 225/564 [58:47<1:25:21, 15.11s/it]

...


❌ Lỗi tại index 301 (DataFrame index = 301):
Chấp hành quy định về tốc độ và khoảng cách giữa các xe được


Sinh trả lời:  56%|█████▌    | 317/564 [1:24:33<1:03:27, 15.42s/it]

DEBUG - raw_text:
 {
  "answer": "Biển số xe ở Việt Nam được phân loại dựa trên màu nền, màu chữ và số, cũng như ký hiệu đặc biệt để phân biệt các đối tượng sử dụng khác nhau.\n\nCác loại biển số xe được phân loại như sau:\n\n*   Biển số xe nền màu xanh, chữ và số màu trắng: Cấp cho xe của cơ quan Đảng, Nhà nước, tổ chức chính trị - xã hội, đơn vị sự nghiệp công lập.\n*   Biển số xe nền màu đỏ, chữ và số màu trắng: Cấp cho xe quân sự.\n*   Biển số xe nền màu vàng, chữ và số màu đen: Cấp cho xe hoạt động kinh doanh vận tải.\n*   Biển số xe nền màu trắng, chữ và số màu đen: Cấp cho xe của các tổ chức, cá nhân trong nước, không thuộc các đối tượng trên.\n*   Biển số xe nền màu trắng, chữ màu đỏ, số màu đen, có ký hiệu “NG”: Cấp cho xe của cơ quan đại diện ngoại giao, cơ quan lãnh sự và nhân viên nước ngoài mang chứng minh thư ngoại giao của cơ quan đó.\n*   Biển số xe nền màu trắng, chữ màu đỏ, số màu đen, có ký hiệu “QT”: Cấp cho xe của cơ quan đại diện của tổ chức quốc tế và nhân viên n

Sinh trả lời: 100%|██████████| 564/564 [2:36:21<00:00, 16.63s/it]  


ValueError: Length of values (559) does not match length of index (564)

In [ ]:
final_output_path = './data/data_processed/final_data_system_response.csv'
df.to_csv(final_output_path, index=False)
print(f" Đã cập nhật và lưu toàn bộ vào: {final_output_path}")

> Configure

In [ ]:
setting=Settings()

> Get embedding gemini 

In [ ]:
import google.generativeai as genai

genai.configure(api_key=gemini.key_manager.get_next_key())

def get_gemini_embedding(texts):
    try :
        result = genai.embed_content(
            model="models/text-embedding-004",
            content=texts,
            task_type="SEMANTIC_SIMILARITY"
        )
        
        return result['embedding']
    except Exception as e:
        print("Đã xảy ra lỗi:", e)
        return [0.0] * 768

> get embedding cohere 

In [ ]:
# import cohere

# def get_cohere_embedding(text: str, model_name: str = "embed-multilingual-v3.0") -> list:
#     try:
#         co = cohere.ClientV2(api_key=cohere_api.key_manager.get_next_key())
#         res = co.embed(
#             texts=[text],
#             model=model_name,
#             input_type="classification",
#             embedding_types=["float"],
#         )a

#         # Trả về nhúng đầu tiên
#         return res.embeddings.float[0]
#     except Exception as e:
#         print("Đã xảy ra lỗi:", e)
#         return [0.0] * 1024  # Trả về danh sách 1024 số 0 nếu có lỗi


>Get  rouge-L & F1 Score

In [ ]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def compute_rouge_l(pred: str, ref: str) -> float:
    try:
        score = scorer.score(ref, pred)
        return score['rougeL'].fmeasure
    except:
        return 0.0

def compute_f1(pred: str, ref: str) -> float:
    pred_tokens = pred.lower().split()
    ref_tokens = ref.lower().split()
    common = set(pred_tokens) & set(ref_tokens)
    if not pred_tokens or not ref_tokens:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(ref_tokens)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

In [ ]:
df = pd.read_csv('./data/data_processed/final_data_system_response.csv')
df = df.dropna(subset=['answer', 'answer_from_gemini_rag_final'])

print(df.shape)

In [ ]:
# # df['answer_embedding_cohere'] = df['answer'].apply(get_cohere_embedding)
# df['answer_from_gemini_embedding_cohere'] = df['answer_from_gemini'].apply(get_cohere_embedding)


df['answer_embedding'] = df['answer'].apply(get_gemini_embedding)
df['answer_from_gemini_rag_final_embedding'] = df['answer_from_gemini_rag_final'].apply(get_gemini_embedding)

df['cosine_similarity_gemini_rag_final'] = df.apply(
    lambda row: cosine_similarity(
        [row['answer_embedding']], 
        [row['answer_from_gemini_rag_final_embedding']]
    )[0][0],
    axis=1
)

# df['cosine_similarity_cohere'] = df.apply(
#     lambda row: cosine_similarity(
#         [row['answer_embedding_cohere']], 
#         [row['answer_from_gemini_embedding_cohere']]
#     )[0][0],
#     axis=1
# )

In [ ]:
df['rouge_l'] = df.apply(lambda row: compute_rouge_l(row['answer_from_gemini_rag_final'], row['answer']), axis=1)
df['f1'] = df.apply(lambda row: compute_f1(row['answer_from_gemini_rag_final'], row['answer']), axis=1)


In [ ]:
print(f" Trung bình Cosine (Gemini): {df['cosine_similarity_gemini_rag_final'].mean():.4f}")
print(f" Trung bình ROUGE-L: {df['rouge_l'].mean():.4f}")
print(f" Trung bình F1 Score: {df['f1'].mean():.4f}") 